# The NFL circle of parity 2022

This notebook contains the code for the Phoebe article *The NFL circle of parity 2022*. 

## Get the data

First we need the outcome of every game in this years NFL season. For this, we take the table from [www.pro-football-reference.com](https://www.pro-football-reference.com/years/2022/games.htm) and save it as a text file.

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

df = pd.read_csv('Games2022.txt')

# delete unused columns and rename others
del df['Unnamed: 5']
df.rename(columns = {'Unnamed: 7':'status','Winner/tie':'Winner','Loser/tie':'Loser'}, inplace = True)

# we are only interested in games that are already played
df = df[df.status=='boxscore']

# we know that each team has won at least once, so this gives us a list of teams
teams = np.unique(df['Winner'])

## Create a Graph

In [ ]:
# we need a directed Graph
G = nx.DiGraph()

# add all teams as nodes
G.add_nodes_from(teams)

# add the edges to the graph
for index,row in df.iterrows():
    # we only want to do this if a team wins
    if row.PtsW>row.PtsL:
        G.add_edge(row.Winner,row.Loser)

We can use the `networkx` function `simple_cycles` to search for cycles. With 32 teams and 167 games played so far, there is an unfathomable number of cycles. If we were to compute all of them, it would take way too long. However we are only interested in a cycle that contains all 32 teams (and there are also many of those). Therefore we simply stop once we find one.

In [ ]:
# find a cycle that contains all 32 teams
cycles = nx.simple_cycles(G)

while True:
    cycle = next(cycles)
    if len(cycle)>31:
        break

print(cycle)

## Plot the circle

We have the result, but it is currently only displayed as a boring list. We can use `matplotlib` to make it a bit more exiting. Instead of writing out the team name, we grab the logos of the teams (from [www.sportytell.com](https://sportytell.com/nfl/nfl-team-logos/)) and display them in a circle according the the previously determined cycle.

In [ ]:
import matplotlib.image as mpimg

# we save the logos in a dictionary
team_logos = {}

for name in teams:
    
    team_name = name.lower().replace(' ','-')
    logo = f'https://i0.wp.com/sportytell.com/wp-content/uploads/2020/11/nfl-{team_name}-team-logo.png?resize=300%2C300&ssl=1'
    
    # not good practice, but a few teams have a slightly different format
    # and it is much quicker to address this problem this way
    try:
        team_logos[name] = mpimg.imread(logo)
    except:
        print(f'problems with {name}')
        
# we just add the logos manually
team_logos['Las Vegas Raiders'] = mpimg.imread('https://i0.wp.com/sportytell.com/wp-content/uploads/2020/11/nfl-oakland-raiders-team-logo.png?resize=300%2C300&ssl=1')
team_logos['Los Angeles Rams'] = mpimg.imread('https://i0.wp.com/sportytell.com/wp-content/uploads/2020/11/los-angeles-rams-logo.png?resize=300%2C300&ssl=1')
team_logos['Miami Dolphins'] = mpimg.imread('https://i0.wp.com/sportytell.com/wp-content/uploads/2020/11/nfl-miami-dolphins-logo018.png?resize=300%2C300&ssl=1')
team_logos['Washington Commanders'] = mpimg.imread('https://i0.wp.com/sportytell.com/wp-content/uploads/2020/11/washington-football-team-logo.png?resize=300%2C300&ssl=1')
       
# we also grab the NFL logo
nfl_logo = 'https://i0.wp.com/sportytell.com/wp-content/uploads/2020/11/nfl-league-logo.png?resize=300%2C300&ssl=1'
nfl_logo_img = mpimg.imread(nfl_logo)

In [ ]:
import matplotlib.pyplot as plt

fig,ax=plt.subplots(figsize=(7,7))
logo_size = 0.16

for i,name in enumerate(cycle):
    
    # place the logs along a circle
    x = np.sin(i/len(cycle)*2*np.pi)-logo_size/2
    y = np.cos(i/len(cycle)*2*np.pi)-logo_size/2
    axins = ax.inset_axes([x,y,logo_size,logo_size],transform=ax.transData)
    axins.imshow(team_logos[name])    
    axins.axis('off')
    
    # create the text with the result of the game
    losing_team = name
    winning_team = cycle[i-1]
    game = df[(df.Winner==winning_team) & (df.Loser==losing_team)].iloc[0]
    text = f'Week {game.Week} \n {game.PtsW:.0f}–{game.PtsL:.0f}'
    # we place the text between the logos
    x = 0.8*np.sin((i-0.5)/len(cycle)*2*np.pi)
    y = 0.8*np.cos((i-0.5)/len(cycle)*2*np.pi)
    ax.text(x,y,text,transform=ax.transData,fontsize=6,
            ha='center',va='center',weight='bold')
    
# put the NFL logo in the center
axins = ax.inset_axes([-0.5,-0.5, 1, 1],transform=ax.transData)
axins.imshow(nfl_logo_img)
axins.axis('off')

# and some final adjustments
ax.axis('off')
ax.set(xlim=[-1,1],ylim=[-1,1])

#plt.savefig('circle-of-parity.png',dpi=400,transparent=False)
plt.show()